# ADMM 算法

### 一致性问题

假设我们需要求解优化问题
$$
\min_{x} \sum_{i=1}^N f_i(x)+g(x)
$$
其中 $g(\cdot)$ 是一个全局的正则项。一个典型的例子是 Lasso
$$
\frac{1}{2}\Vert Ax-b\Vert^2+\lambda \Vert x\Vert_1.
$$

它可以转化成 ADMM 一致性问题
$$\begin{align*}
\min_{x_1,\ldots,x_N,z}\  & f_i(x_i)+g(z)\\
\mathrm{s.t.}\  & x_i=z,i=1,\ldots,N.
\end{align*}$$

迭代公式为
$$
\begin{align*}
x_i^{k+1} & =\underset{x_i}{\arg\min}\ f_i(x_i)+\frac{\rho}{2}\Vert x_i-z^{k}+u_i^{k}\Vert^{2}\\
z^{k+1} & =\underset{z}{\arg\min}\ g(z)+\frac{N\rho}{2}\Vert z-\bar{x}^{k+1}-\bar{u}^{k}\Vert^{2}\\
u_i^{k+1} & =u_i^{k}+x_i^{k+1}-z^{k+1}.
\end{align*}
$$

注意其中带下标 $i$ 的更新要对所有 $i=1,\ldots,N$ 进行。

总的原问题残差范数为
$$
\Vert r^{k+1}\Vert=\sqrt{\sum_{i=1}^N \Vert x_i^{k+1} - z^{k+1}\Vert^2},
$$

总的对偶问题残差范数为
$$
\Vert s^{k+1}\Vert=\rho \sqrt{N} \Vert z^{k+1} - z^k\Vert.
$$

### Lasso

对于 Lasso 问题，假设数据按行分块，

$$
A_{n\times p}=\left[\begin{array}{c}
A_{1}\in\mathbb{R}^{n_{1}\times p}\\
\vdots\\
A_{N}\in\mathbb{R}^{n_{N}\times p}
\end{array}\right],\quad b_{n\times1}=\left[\begin{array}{c}
b_{1}\in\mathbb{R}^{n_{1}}\\
\vdots\\
b_{N}\in\mathbb{R}^{n_{N}}
\end{array}\right].
$$

其迭代公式为

$$
\begin{align*}
x_i^{k+1} & =(A_i'A_i+\rho I)^{-1}(A_i'b_i+\rho(z^{k}-u_i^{k}))\\
z^{k+1} & =S_{\lambda/(\rho N)}(\bar{x}^{k+1}+\bar{u}^{k})\\
u_i^{k+1} & =u_i^{k}+x_i^{k+1}-z^{k+1}.
\end{align*}
$$

### PySpark 实现

In [ ]:
import os
import numpy as np
np.set_printoptions(linewidth=100)

先生成模拟数据：

In [ ]:
np.random.seed(123)
n = 100000
p = 100
nz = 20
A = np.random.normal(size=(n, p))
# 真实的 x 只有前20个元素非零，其余均为0
xtrue = np.random.normal(size=nz)
xtrue = np.concatenate((xtrue, np.zeros(p - nz)))
b = A.dot(xtrue) + np.random.normal(size=n)
dat = np.hstack((b.reshape(n, 1), A))
if not os.path.exists("data"):
    os.makedirs("data", exist_ok=True)
np.savetxt("data/lasso.txt", dat, fmt="%.9f", delimiter="\t")

配置和启动 PySpark：

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
# 本地模式
spark = SparkSession.builder.\
    master("local[*]").\
    config("spark.executor.memory", "2g").\
    config("spark.driver.memory", "2g").\
    appName("ADMM").\
    getOrCreate()
sc = spark.sparkContext
# sc.setLogLevel("ERROR")
print(spark)
print(sc)

PySpark 读取文件：

In [ ]:
file = sc.textFile("data/lasso.txt")

# 打印前5行，并将每行字符串截尾
text = file.map(lambda x: x[:70] + "...").take(5)
print(*text, sep="\n")

file.getNumPartitions()

分区转换：

In [ ]:
# str => np.array
def str_to_vec(line):
    # 分割字符串
    str_vec = line.split("\t")
    # 将每一个元素从字符串变成数值型
    num_vec = map(lambda s: float(s), str_vec)
    # 创建 Numpy 向量
    return np.fromiter(num_vec, dtype=float)

# Iter[str] => Iter[matrix]
def part_to_mat(iterator):
    # Iter[str] => Iter[np.array]
    iter_arr = map(str_to_vec, iterator)

    # Iter[np.array] => list(np.array)
    dat = list(iter_arr)

    # list(np.array) => matrix
    if len(dat) < 1:  # Test zero iterator
        mat = np.array([])
    else:
        mat = np.vstack(dat)

    # matrix => Iter[matrix]
    yield mat

dat = file.mapPartitions(part_to_mat).filter(lambda x: x.shape[0] > 0)
dat.cache()
print(dat.count())

创建一个新的 RDD，用来存储每个数据分区上的 $x_i$ 和 $u_i$ 变量：

In [ ]:
def create_xu(block_dat):
    p = block_dat.shape[1] - 1
    xi = np.zeros(p)
    ui = np.zeros(p)
    return xi, ui

params = dat.map(create_xu)
params.first()

在后续更新中，我们会将数据 RDD（dat）和参数 RDD（params）进行拼接，从而更新参数。

In [ ]:
dat.zip(params).first()

在主进程上设定若干变量，包括初始化 $z$：

In [ ]:
# 样本量
n = dat.map(lambda x: x.shape[0]).reduce(lambda x, y: x + y)
print(f"n = {n}")

# 变量数
p = dat.first().shape[1] - 1
print(f"p = {p}")

# 分区数
N = dat.count()
print(f"N = {N}")

# rho
rho = 10.0

# lambda
lam = 0.001 * n

# z变量
z = np.zeros(p)

更新一次 $x_i$：

In [ ]:
def update_x(block_dat_and_param, z, rho):
    block_dat, (xi, ui) = block_dat_and_param
    # 分离因变量向量和自变量矩阵
    bi = block_dat[:, 0]
    Ai = block_dat[:, 1:]
    p = Ai.shape[1]
    # 计算新 xi
    xi_new = np.linalg.solve(Ai.T.dot(Ai) + rho * np.eye(p), Ai.T.dot(bi) + rho * (z - ui))
    return xi_new, ui

updated_x = dat.zip(params).map(lambda d: update_x(d, z, rho))
updated_x.first()

计算 $\bar{x}$ 和 $\bar{u}$：

In [ ]:
xbar, ubar = updated_x.reduce(lambda xu1, xu2: (xu1[0] + xu2[0], xu1[1] + xu2[1]))
xbar /= N
ubar /= N
print(xbar)
print(ubar)

更新一次 $z$：

In [ ]:
def soft_thresholding(a, k):
    return np.sign(a) * np.maximum(0.0, np.abs(a) - k)

znew = soft_thresholding(xbar + ubar, lam / (rho * N))
znew

更新一次 $u_i$：

In [ ]:
def update_u(param, z):
    xi_new, ui = param
    # 计算新 ui
    ui_new = ui + xi_new - z
    return xi_new, ui_new

updated_u = updated_x.map(lambda d: update_u(d, znew))
updated_u.first()

计算原问题残差范数

$$
\Vert r^{k+1}\Vert=\sqrt{\sum_{i=1}^N \Vert x_i^{k+1} - z^{k+1}\Vert^2}.
$$

In [ ]:
# 在每个分块上计算 ||xi_new - z||^2，然后汇总求和
resid_r_norm = updated_u.map(lambda d: np.sum(np.square(d[0] - znew))).reduce(lambda x, y: x + y)
resid_r_norm = np.sqrt(resid_r_norm)
print(resid_r_norm)

计算对偶问题残差范数：

In [ ]:
resid_s_norm = rho * np.sqrt(N) * np.linalg.norm(znew - z)
print(resid_s_norm)

接下来将整个过程写入一个循环，同时设定最大迭代次数为100，收敛的阈值为0.001。

In [ ]:
max_iter = 100
tol = 0.001

for i in range(max_iter):
    # x 更新
    updated_x = dat.zip(params).map(lambda d: update_x(d, z, rho))
    updated_x.cache()
    # 计算平均
    xbar, ubar = updated_x.reduce(lambda xu1, xu2: (xu1[0] + xu2[0], xu1[1] + xu2[1]))
    xbar /= N
    ubar /= N
    # z 更新
    znew = soft_thresholding(xbar + ubar, lam / (rho * N))
    # u 更新
    updated_u = updated_x.map(lambda d: update_u(d, znew))
    updated_u.cache()
    # 计算残差大小
    resid_r_norm = updated_u.map(lambda d: np.sum(np.square(d[0] - znew))).reduce(lambda x, y: x + y)
    resid_r_norm = np.sqrt(resid_r_norm)
    resid_s_norm = rho * np.sqrt(N) * np.linalg.norm(znew - z)
    # 更新 x、z 和 u 的取值
    params = updated_u
    params.cache()
    z = znew
    # 打印残差信息，判断是否收敛
    if i % 1 == 0:
        print(f"Iteration {i}, ||r|| = {resid_r_norm:.6f}, ||s|| = {resid_s_norm:.6f}")
    if resid_r_norm <= tol and resid_s_norm <= tol:
        break

In [ ]:
z

In [ ]:
xtrue

In [ ]:
sc.stop()